# Activity Classification - DNN2 Training

This notebook trains our second Deep Neural Network on the physical activity dataset.


## Load Data and Prepare Training Set

In [28]:
%reset -f

import importlib
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import activity_functions
importlib.reload(activity_functions)

from activity_functions import *

In [29]:
activtity = load_data()

Loaded from Kaggle: /home/thuy/.cache/kagglehub/datasets/diegosilvadefrana/fisical-activity-dataset/versions/4/dataset2.csv


In [ ]:
df_train, df_test = create_train_test(activtity, test_ratio=0.2)

(2291244, 33)
(572812, 33)


In [31]:
X_train, y_train, X_test, y_test = prepare_for_train(df_train, df_test)

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

## Hyperband + Regularization

In [ ]:
tuner = kt.Hyperband(
    build_model_cnn,
    objective="val_accuracy",
    max_epochs=50,
    factor=3,
    directory="dnn2",
    project_name="hyperband_tuning"
)

tuner.search(
    X_train,
    y_train_encoded,
    validation_split=0.2,
    epochs=50,
    batch_size=64,
    verbose=1
)

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best hyperparameters:")
for hp_name in best_hps.values.keys():
    print(f"  {hp_name}: {best_hps.get(hp_name)}")


## Fit Best Model

In [ ]:
best_model = tuner.hypermodel.build(best_hps)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=7,
    restore_best_weights=True
)

history = best_model.fit(
    X_train,
    y_train_encoded,
    validation_split=0.2,
    epochs=50,
    batch_size=64,
    callbacks=[early_stop],
    verbose=1
)

#### Accuracy Plot

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training vs Validation Accuracy")
plt.legend(["Train", "Validation"])
plt.show()

#### Loss Plot

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.legend(["Train", "Validation"])
plt.show()


## Model Evaluation

In [ ]:
y_test_hat = best_model.predict(X_test)
y_test_pred = np.argmax(y_test_hat, axis=1)
compute_scores(y_test_encoded, y_test_pred, verbose=True)

## Summary

Discuss results